In [842]:
import numpy as np
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 1000000)
pd.set_option('display.max_rows', 1000000)



In [843]:
#Importing data and removing empty columns
data = pd.read_table('Fsatellites.tsv', encoding = "ISO-8859-1")
print(data.shape)

for col in data.columns:
    if 'Unnamed' in col:
        del data[col]
        
for col in data.columns:
    if 'Source' in col:
        del data[col]
data.head()

(1381, 254)


,"Name of Satellite, Alternate Names",Country/Org of UN Registry,Country of Operator/Owner,Operator/Owner,Users,Purpose,Detailed Purpose,Class of Orbit,Type of Orbit,Longitude of GEO (degrees),Perigee (km),Apogee (km),Eccentricity,Inclination (degrees),Period (minutes),Launch Mass (kg.),Dry Mass (kg.),Power (watts),Date of Launch,Expected Lifetime,Contractor,Country of Contractor,Launch Site,Launch Vehicle,COSPAR Number,NORAD Number,Comments
0,AAUSat-5 (Aalborg University Cubesat 5),NR (12/15),Denmark,Aalborg University,Civil,Communications,Automatic Identification System (AIS),LEO,Non-Polar Inclined,0.00,395,409,0.001030,51.64,92.6,1,NaN,NaN,10/2/2015,NaN,Aalborg University,Denmark,International Space Station,Nanorack Deployer,1998-067GZ,40948,Main goal is to test a AIS receiver built by s...
1,"ABS-2 (Koreasat-8, ST-3)",NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,"35,778","35,793",0.000178,0.08,1436.03,"6,330",NaN,"16,000",2/6/2014,15 yrs.,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA,2014-006A,39508,"32 C-band, 51 Ku-band, and 6 Ka-band transpond..."
2,"ABS-3 (Agila 2, Mabuhay 1)",Philippines,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,146.06,"35,769","35,802",0.000391,0.05,1436.07,"3,775","1,800","9,000",8/19/1997,15 yrs.,Space Systems/Loral,USA,Xichang Satellite Launch Center,Long March CZ3B,1997-042A,24901,Most powerful telecommunications satellite in ...
3,ABS-3A,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,-3.00,"35,788","35,803",0.000178,0.10,1436,"2,000",NaN,NaN,3/2/2015,15 yrs.,Boeing Satellite Systems,NaN,Cape Canaveral,Falcon 9,2015-010A,40424,Coverage of Americas Europe and Africa.
4,"ABS-4 (ABS-2i, MBSat, Mobile Broadcasting Sate...",NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,"35,780","35,793",0.000154,0.01,1436.1,"4,143","1,700","7,400",3/13/2004,12 yrs.,Space Systems/Loral,USA,Cape Canaveral,Atlas 3,2004-007A,28184,Purchased by ABS in 2013.


In [844]:
#Renaming columns
data.columns.values

array(['Name of Satellite, Alternate Names', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments'], dtype=object)

In [845]:
data.columns=['name', 'country_reg', 'country', 'owner', 'users',
              'purpose', 'description', 'CLO', 'TOO', 'longitude',
              'perigee_km', 'apogee_km', 'eccentricity', 'inclination', 
              'period_minutes', 'launch_mass', 'dry_mass', 'power_watts',
              'launch_date', 'expected_lifetime_years', 'contractor', 
              'contractor_country', 'launch_site', 'launch_vehicle',
              'COSPAR', 'NORAD', 'comments']


In [846]:
data['CLO'] = data['CLO'].replace('LEO ', 'LEO')

In [847]:
#Standardizing expected_lifetime values
data_NoC = data.replace(to_replace=',', value='', regex=True)
data_NoC['expected_lifetime_years'] = data_NoC['expected_lifetime_years'].replace(to_replace="(yrs\.|yr\.|hrs\.|trs)", value='', regex=True)
data_NoC['expected_lifetime_years'] = data_NoC['expected_lifetime_years'].replace(to_replace="\.?[0-9]*-", value='', regex=True)
data_NoC['expected_lifetime_years'] = data_NoC['expected_lifetime_years'].replace(to_replace="\+", value='', regex=True)
pd.unique(data_NoC['expected_lifetime_years'])

array([nan, '15 ', '12 ', '14 ', '2 ', '3 ', '1 ', '5 ', '10 ', '14', '8 ',
       '7 ', '1.5 ', '13 ', '6 ', '.5 ', '9 ', '18 ', '16 ', '.25 ', '11 ',
       '15  ', '17 ', '30 ', '14.5 ', '7.25 ', ' 3 ', '4 ', '11.5 ',
       '7.5 ', '2.5 ', '12.6 '], dtype=object)

In [848]:
data['expected_lifetime_years'].isnull().sum()

428

In [849]:
data_NoC['expected_lifetime_years'].isnull().sum()

428

In [850]:
#Convert data
data_converted = data_NoC.convert_objects(convert_dates=True, convert_numeric=True,)
data = data_converted


/Users/ianssmith/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app


In [851]:
data['launch_date'] = pd.to_datetime(data['launch_date'])
data['expected_lifetime_years'].isnull().sum()

428

In [852]:
pd.unique(data.expected_lifetime_years)

array([   nan,  15.  ,  12.  ,  14.  ,   2.  ,   3.  ,   1.  ,   5.  ,
        10.  ,   8.  ,   7.  ,   1.5 ,  13.  ,   6.  ,   0.5 ,   9.  ,
        18.  ,  16.  ,   0.25,  11.  ,  17.  ,  30.  ,  14.5 ,   7.25,
         4.  ,  11.5 ,   7.5 ,   2.5 ,  12.6 ])

In [853]:
#Check for correct dtype
datatypes = data.columns.to_series().groupby(data.dtypes).groups
datatypes

{dtype('int64'): ['perigee_km', 'apogee_km', 'NORAD'],
 dtype('float64'): ['longitude',
  'eccentricity',
  'inclination',
  'period_minutes',
  'launch_mass',
  'dry_mass',
  'power_watts',
  'expected_lifetime_years'],
 dtype('<M8[ns]'): ['launch_date'],
 dtype('O'): ['name',
  'country_reg',
  'country',
  'owner',
  'users',
  'purpose',
  'description',
  'CLO',
  'TOO',
  'contractor',
  'contractor_country',
  'launch_site',
  'launch_vehicle',
  'COSPAR',
  'comments']}

In [854]:
#Create columns to isolate launch year/month/day
data['launch_year'], data['launch_month'], data['launch_day'] = data['launch_date'].dt.year, data['launch_date'].dt.month, data['launch_date'].dt.day
data.columns



Index(['name', 'country_reg', 'country', 'owner', 'users', 'purpose',
       'description', 'CLO', 'TOO', 'longitude', 'perigee_km', 'apogee_km',
       'eccentricity', 'inclination', 'period_minutes', 'launch_mass',
       'dry_mass', 'power_watts', 'launch_date', 'expected_lifetime_years',
       'contractor', 'contractor_country', 'launch_site', 'launch_vehicle',
       'COSPAR', 'NORAD', 'comments', 'launch_year', 'launch_month',
       'launch_day'],
      dtype='object')

In [855]:
#Remove nickaname from name column to its own column
data['nickname'] = data['name'].str.extract('(\(.*\))')
data['name'] = data['name'].replace(to_replace="\(.*\)", value='', regex=True)


In [856]:
#Fix and standardize longitudinal degrees to 360 for GEO orbits
data['longitude'][data['longitude'] > 180] = data['longitude'] - 360
data['longitude'][data['longitude'] < 0 ] = data['longitude'] + 360

/Users/ianssmith/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/ianssmith/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [857]:
#Adding arbitrary longitudes to non-GEO orbits for successional calculations
##GEOsynchronous orbits appear stationary at certian longitudes from earths surface
data['longitude'][data['longitude'] == 0 ] = data['longitude'].apply(lambda v: np.random.randint(360))

/Users/ianssmith/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [858]:
#Calculate radians for GEO orbit longitudes
data['radians'] = np.deg2rad(data['longitude'])

In [859]:
##Create constant variables
#(mean)radius of earth km
earthRad = 6371
#Mass of earth times gravitational constant
GM = 398600.4


In [860]:
#Include earth's radius in apogee and perigee values for elliptical calculations
data['apogeeR'] = data['apogee_km'] + 6371
data['perigeeR'] = data['perigee_km'] + 6371

In [861]:
#Calculate the major/semimajor axis of orbits
data['majorA'] = data.apogeeR + data.perigeeR
data['semimajorA'] = data['majorA']/2


In [862]:
#The distance between earth and the empty focus of the ellipse
data['fDistance'] = data['majorA'] - data['perigeeR']*2

In [863]:
#Calculate the minor/semiminor axis of orbits
data['minorA'] = np.sqrt((data['majorA'])**2 - (data['fDistance'])**2)
data['semiminorA'] = data['minorA']/2

In [864]:
#Mean motion of orbits
data['meanmotion'] = np.sqrt(GM/(data['semimajorA']**3))

In [865]:
#Finding the period in seconds
data['period_seconds'] = (2*np.pi)*(np.sqrt((data['semimajorA']**3)/(GM))) #gives more precise values
data['simple_seconds'] = data['period_minutes']*60
print("Radian approach\n")
print(data.period_seconds.head())
print("\nConverting from data\n")
print(data.simple_seconds.head())
print("\n Radian conversion gives more accurate figures")

Radian approach

0     5547.312305
1    86140.586331
2    86140.586331
3    86171.238435
4    86143.651378
Name: period_seconds, dtype: float64

Converting from data

0     5556.0
1    86161.8
2    86164.2
3    86160.0
4    86166.0
Name: simple_seconds, dtype: float64

 Radian conversion gives more accurate figures


In [866]:
#Calculating theoretical time  until longitudinal location for GEO orbits
print('Calculating directly via radians\n')
data['T_long'] = data['radians']*np.sqrt((data['semimajorA']**3)/GM)
print(data.T_long.head())

print("\n\nCalculating per degree longitude\n")
data['sec_per_longdegree'] = data['period_seconds']/360
print("\nSeconds/degree traveled")
print(data.sec_per_longdegree.head())
print("\nTimes longitudinal location")
data['time_till_longitude'] = data['longitude']*data['sec_per_longdegree']
print(data.time_till_longitude.head())

print("\nNo difference in accuracy")

Calculating directly via radians

0     3929.346216
1    17945.955486
2    34949.150110
3    85453.144782
4    17946.594037
Name: T_long, dtype: float64


Calculating per degree longitude


Seconds/degree traveled
0     15.409201
1    239.279406
2    239.279406
3    239.364551
4    239.287920
Name: sec_per_longdegree, dtype: float64

Times longitudinal location
0     3929.346216
1    17945.955486
2    34949.150110
3    85453.144782
4    17946.594037
Name: time_till_longitude, dtype: float64

No difference in accuracy


In [905]:
#Calculate mean anomaly at position X
data['meanAnom'] = data['meanmotion'] * data['time_till_longitude']

In [906]:
#Distance of earth from center of ellipse 
data['Efromcenter'] = data['semimajorA'] - data['perigeeR']

In [907]:
#Test function 
#def eccen(meanAnom, eccentricity):
#     ea = meanAnom - ((meanAnom-(eccentricity*np.sin(meanAnom))-meanAnom)/(1-eccentricity*np.cos(meanAnom)))
#     difference = ea - meanAnom
#     if (difference < 0.0000001):
#         return ea
#     else:
#         eccen(ea, eccentricity)

In [908]:
# eccen(5.87286, 0.0501)# 0.8501)

In [909]:
#Calculate eccentric Anomaly at position X
def eccentricAnom(meanAnom, eccentricity):
    difference = pd.DataFrame()
    eccentricAnom = meanAnom - ((meanAnom-(eccentricity*np.sin(meanAnom))-meanAnom)/(1-eccentricity*np.cos(meanAnom)))
    difference = eccentricAnom - meanAnom
    for item, frame in difference.iteritems():
        if (frame < 0.000000):
            return eccentricAnom
        else:
            eccenTest(eccentricAnom, eccentricity)
            
#data.meanAnom.apply(eccentest, axis=1)

In [910]:
data['eccentricAnom'] = eccentricAnom(data['meanAnom'], data['eccentricity'])

In [912]:
#checking 
data.loc[780]

name                                                              Mercury 2 
country_reg                                                              USA
country                                                                  USA
owner                              National Reconnaissance Office (NRO)/USAF
users                                                               Military
purpose                                                    Earth Observation
description                                          Electronic Intelligence
CLO                                                                      GEO
TOO                                                                      NaN
longitude                                                             336.49
perigee_km                                                             33674
apogee_km                                                              37900
eccentricity                                                          0.0501

In [913]:
#Calculate true Anomaly  at position X
data['trueAnom'] = np.arccos((np.cos(data['eccentricAnom'])-data['eccentricity'])/(1 - data['eccentricity']*np.cos(data['eccentricAnom'])))

In [914]:
#Adjusting for 360 degrees
data['trueAnomadj'] = data['trueAnom']*2

In [915]:
#Calculating flight path  at position X
data['flightpath'] = np.arctan(data['eccentricity']*np.sin(data['trueAnom'])/(1 + data['eccentricity']*np.cos(data['trueAnom'])))

In [916]:
#Calculating altitude at position X
data['distanceAt'] = (data['semimajorA'] * (1-data['eccentricity']*np.cos(data['eccentricAnom'])))-6371

In [917]:
#Converting to cartesian coordiantes
data['cartX'] = data['semimajorA']*np.cos(data['trueAnomadj']) #- data['Efromcenter']
data['cartY'] = data['semiminorA']*np.sin(data['trueAnomadj'])

In [926]:
data.head()

,name,country_reg,country,owner,users,purpose,description,CLO,TOO,longitude,perigee_km,apogee_km,eccentricity,inclination,period_minutes,launch_mass,dry_mass,power_watts,launch_date,expected_lifetime_years,contractor,contractor_country,launch_site,launch_vehicle,COSPAR,NORAD,comments,launch_year,launch_month,launch_day,nickname,radians,apogeeR,perigeeR,majorA,semimajorA,fDistance,minorA,semiminorA,meanmotion,period_seconds,simple_seconds,T_long,sec_per_longdegree,time_till_longitude,meanAnom,Efromcenter,eccentricAnom,trueAnom,trueAnomadj,trueAnomadj2,flightpath,distanceAt,cartX,cartY
0,AAUSat-5,NR (12/15),Denmark,Aalborg University,Civil,Communications,Automatic Identification System (AIS),LEO,Non-Polar Inclined,255.00,395,409,0.001030,51.64,92.60,1,NaN,NaN,2015-10-02,NaN,Aalborg University,Denmark,International Space Station,Nanorack Deployer,1998-067GZ,40948,Main goal is to test a AIS receiver built by s...,2015,10,2,(Aalborg University Cubesat 5),4.450590,6780,6766,13546,6773.0,14,13545.992765,6772.996383,0.001133,5547.312305,5556.0,3929.346216,15.409201,3929.346216,4.450590,7.0,4.449595,1.834585,3.669170,6.284180,0.000995,403.812272,-5852.071210,-3409.806320
1,ABS-2,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,35778,35793,0.000178,0.08,1436.03,6330,NaN,16000,2014-02-06,15,Space Systems/Loral,USA,Guiana Space Center,Ariane 5 ECA,2014-006A,39508,32 C-band 51 Ku-band and 6 Ka-band transponder...,2014,2,6,(Koreasat-8 ST-3),1.308997,42164,42149,84313,42156.5,15,84312.998666,42156.499333,0.000073,86140.586331,86161.8,17945.955486,239.279406,17945.955486,1.308997,7.5,1.309169,1.309341,2.618682,2.618510,0.000172,35783.559105,-36523.088472,21053.134843
2,ABS-3,Philippines,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,146.06,35769,35802,0.000391,0.05,1436.07,3775,1800,9000,1997-08-19,15,Space Systems/Loral,USA,Xichang Satellite Launch Center,Long March CZ3B,1997-042A,24901,Most powerful telecommunications satellite in ...,1997,8,19,(Agila 2 Mabuhay 1),2.549228,42173,42140,84313,42156.5,33,84312.993542,42156.496771,0.000073,86140.586331,86164.2,34949.150110,239.279406,34949.150110,2.549228,16.5,2.549446,2.549664,5.099329,5.099110,0.000218,35799.176838,15908.013961,-39039.791887
3,ABS-3A,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,357.00,35788,35803,0.000178,0.10,1436.00,2000,NaN,NaN,2015-03-02,15,Boeing Satellite Systems,NaN,Cape Canaveral,Falcon 9,2015-010A,40424,Coverage of Americas Europe and Africa.,2015,3,2,NaN,6.230825,42174,42159,84333,42166.5,15,84332.998666,42166.499333,0.000073,86171.238435,86160.0,85453.144782,239.364551,85453.144782,6.230825,7.5,6.230816,0.052379,0.104757,6.283195,0.000009,35788.004653,41935.343194,4409.162372
4,ABS-4,NR,Multinational,Asia Broadcast Satellite Ltd.,Commercial,Communications,NaN,GEO,NaN,75.00,35780,35793,0.000154,0.01,1436.10,4143,1700,7400,2004-03-13,12,Space Systems/Loral,USA,Cape Canaveral,Atlas 3,2004-007A,28184,Purchased by ABS in 2013.,2004,3,13,(ABS-2i MBSat Mobile Broadcasting Satellite Ha...,1.308997,42164,42151,84315,42157.5,13,84314.998998,42157.499499,0.000073,86143.651378,86166.0,17946.594037,239.287920,17946.594037,1.308997,6.5,1.309146,1.309294,2.618589,2.618440,0.000149,35784.820614,-36522.002194,21057.021428


In [918]:
data.to_csv('CSatellites.csv')

In [919]:
#Split data by type of orbit
LEO = data[data.CLO.isin(["LEO"])] #Low Earth orbit
GEO = data[data.CLO.isin(["GEO"])] #Geosynchronous
MEO_Elliptical = data[data.CLO.isin(["MEO", "Elliptical"])] #Medium earth orbit and Elliptical 





In [920]:
#Move cislunar satellite into Elliptical MEO group
cis = data[data.TOO.isin(["Cislunar"])]
frames = [MEO_Elliptical, cis]
MEO_Elliptical = pd.concat(frames)

In [921]:
MEO_Elliptical.TOO.value_counts()

Non-Polar Inclined       80
Molniya                  15
Equatorial               12
Deep Highly Eccentric     9
Cislunar                  1
Name: TOO, dtype: int64

In [922]:
#Split Low earth orbit into sun-synchronous and other
LEO_sun = LEO[LEO.TOO.isin(["Sun-Synchronous"])]

In [923]:
LEO_other = LEO[LEO.TOO.isin(["Non-Polar Inclined", "Polar", "Equatorial", "Elliptical"])]

In [924]:
LEO_sun.to_csv("LEO_sun.csv")
LEO_other.to_csv("LEO_other.csv")
GEO.to_csv("GEO.csv")
MEO_Elliptical.to_csv("MEO_Elliptical.csv")

In [925]:
LEO_other.launch_year.value_counts()

2015    86
2013    50
1998    48
1997    38
2014    24
2011    22
1999    21
2012    19
2010    19
2007    17
2002    16
2008     9
2006     8
2005     7
2009     6
2001     5
2003     4
2004     4
2000     2
1996     1
1993     1
1990     1
1989     1
1974     1
Name: launch_year, dtype: int64